In [0]:
import keras
from keras.optimizers import RMSprop
import gym
import base64
from gym.wrappers import Monitor
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import time
from gym import logger as gymlogger
from keras.models import load_model
import random
import math
import time
import glob
import io
from skimage.color import rgb2gray
from IPython.display import HTML
from IPython import display as ipythondisplay
from keras import backend as K
from keras.callbacks import TensorBoard
import os.path
from os import path
from collections import deque 
from datetime import datetime
from keras.models import clone_model
from skimage.transform import resize
from google.colab import files

In [43]:
observe_step_num = 10000
batch_size = 32 
gamma = 0.99 
memreplay = 350000 
episodes = 100 
learnrate = 0.01
eps0 = 1.0 
epsfinal = 0.1 
epsstep = 1000000 
target_num = 10000 
opsteps = 30 
train_dir = "training_dir" 
tensor_dir = "logs" 
file_path = "/content/SpaceInvaders_20191126023715.h5" 
model_restore = "/content/SpaceInvaders_20191126023715.h5" 
testepi = 1 
resume = False
!mkdir training_dir 

mkdir: cannot create directory ‘training_dir’: File exists


In [0]:
def preprocess(observe):
    process = np.uint8(resize(rgb2gray(observe), (84 , 84), mode='constant') * 255)
    return process

In [0]:
input_atari = (84, 84, 4) 
size = 4
def model_atari():
    frames = keras.layers.Input(input_atari, name='frames')
    actions = keras.layers.Input((size,), name='mask')
    normalize = keras.layers.Lambda(lambda x: x / 255.0)(frames)
    conv1 = keras.layers.Conv2D(32, (8, 8), strides=4, activation='relu', name="Conv1")(normalize)
    conv2 = keras.layers.Conv2D(64, (4, 4), strides=2, activation='relu', name="Conv2")(conv1)
    conv3 = keras.layers.Conv2D(64, (3, 3), strides=1, activation='relu', name="Conv3")(conv2)
    conv_flat = keras.layers.core.Flatten(name="flatten")(conv3) 
    hid_layer = keras.layers.Dense(512, activation='relu', name="Dense512")(conv_flat)
    outlayer = keras.layers.Dense(size, name="Output")(hid_layer)
    outfilt = keras.layers.Multiply(name='QValue')([outlayer, actions])
    model = keras.models.Model(input=[frames, actions], output=outfilt)
    optimize = keras.optimizers.RMSprop(lr=0.01, rho=0.95, epsilon=0.01)
    model.compile(optimize, loss=lossfunc)
    model.summary()
    return model

In [0]:
def lossfunc(y, q):
    err = K.abs(y - q)
    quad = K.clip(err, 0.0, 1.0)
    lin = err - quad
    loss = K.mean(0.5 * K.square(quad) + lin)
    return loss  

In [0]:
def action(hist, epsilon, step, model):
    if np.random.rand() <= epsilon or step <= observe_step_num:
        return random.randrange(size)
    else:
        q_val = model.predict([hist, np.ones(size).reshape(1, size)])
        return np.argmax(q_val[0])

def memory_store(memory, hist, action, reward, next_hist, dead):
    memory.append((hist, action, reward, next_hist, dead))


def get_one_hot(targets, nb_classes):
    return np.eye(nb_classes)[np.array(targets).reshape(-1)]

def train_batch(memory, model_target, model):
    small_batch = random.sample(memory, batch_size)
    
    hist = np.zeros((batch_size, input_atari[0],
                        input_atari[1], input_atari[2]))
    next_hist = np.zeros((batch_size, input_atari[0],
                             input_atari[1], input_atari[2]))
    target = np.zeros((batch_size,))
    action, reward, dead = [], [], []

    for index, value in enumerate(small_batch):
        hist[index] = value[0]
        next_hist[index] = value[3]
        action.append(value[1])
        reward.append(value[2])
        dead.append(value[4])

    action_mask = np.ones((batch_size, size))
    next_Q_values = model_target.predict([next_hist, action_mask])
    for i in range(batch_size):
        if dead[i]:
            target[i] = -1
        else:
            target[i] = reward[i] + gamma * np.amax(next_Q_values[i])

    action_one_hot = get_one_hot(action, ACTION_SIZE)
    target_one_hot = action_one_hot * target[:, None]

    h = model.fit(
        [hist, action_one_hot], target_one_hot, epochs=1,
        batch_size=batch_size, verbose=0)

    return h.history['loss'][0]
 
def show_video():
  l1 = glob.glob('video/*.mp4')
  if len(l1) > 0:
    mp4 = l1[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("The video could not be found")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [48]:
def train():
    env = gym.make("SpaceInvaders-v0")
    step_global = 0
    memory = deque(maxlen=memreplay)
    episode_number = 0
    epsilon = eps0
    epsdecay = (eps0 - epsfinal) / epsstep
    if resume:
        model = load_model(file_path, custom_objects={'lossfunc': lossfunc})
        epsilon = 0.1
    else:
        model = model_atari()

    targetmodel = clone_model(model)
    targetmodel.set_weights(model.get_weights())
    episode_list =[]
    rewards_list = []
    loss_list = []

    while episode_number < episodes:

        done = False
        dead = False
        episode_list.append(episode_number)
        step, score, start_life = 0, 0, 3
        loss = 0.0
        observe = env.reset()
        for _ in range(random.randint(1, opsteps)):
            observe, _, _, _ = env.step(1)
        states = preprocess(observe)
        history = np.stack((states, states, states, states), axis=2)
        history = np.reshape([history], (1, 84, 84, 4))

        while not done:
            actions = action(history, epsilon, step_global, model)
            if epsilon > epsfinal and step_global > observe_step_num:
                epsilon -= epsdecay

            observe, reward, done, info = env.step(actions)
            next_state = preprocess(observe)
            next_state = np.reshape([next_state], (1, 84, 84, 1))
            next_history = np.append(next_state, history[:, :, :, :3], axis=3)
            
            if start_life > info['ale.lives']:
              dead = True
              start_life = info['ale.lives']
            memory_store(memory, history, actions, reward, next_history, dead)  
            if step_global > observe_step_num:
                loss = loss + train_memory_batch(memory, targetmodel, model)
                
                if step_global % target_num == 0:
                    targetmodel.set_weights(model.get_weights())
                    print ("Refreshing the target model")

            score += reward
            history = next_history
            step_global += 1
            step += 1
            if done:
                if step_global <= observe_step_num:
                    state = "observe"
                elif observe_step_num < step_global <= observe_step_num + epsstep:
                    state = "explore"
                else:
                    state = "train"
                print('Episode: {}, State: {},  Score: {}, Average Loss: {}, Step: {}'
                      .format(episode_number, state, score, loss / float(step), step))
                rewards_list.append(score)
                loss_list.append(loss)

                if episode_number % 15 == 0 or (episode_number + 1) == episodes:
                    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
                    file_name = "SpaceInvaders_{}.h5".format(now)
                    model_path = os.path.join(train_dir, file_name)
                    model.save(model_path)

                episode_number += 1    
train()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("QV...)`
  


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
frames (InputLayer)             (None, 84, 84, 4)    0                                            
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 84, 84, 4)    0           frames[0][0]                     
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 20, 20, 32)   8224        lambda_4[0][0]                   
__________________________________________________________________________________________________
Conv2 (Conv2D)                  (None, 9, 9, 64)     32832       Conv1[0][0]                      
____________________________________________________________________________________________

KeyboardInterrupt: ignored

In [49]:
def test():
    env = wrap_env(gym.make('SpaceInvaders-v0'))
    episode_number = 0
    epsilon = 0.001
    global_step = 0
    if path.exists(model_restore):
      model = load_model(model_restore, custom_objects={'huber_loss': lossfunc}) 
    else:
      fil = files.upload()
      ll = list(fil)
      restore_path = os.path.abspath(ll[0])
      model = load_model(restore_path, custom_objects={'huber_loss': lossfunc})
    while episode_number < testepi:
        done = False
        dead = False        
        score, start_life = 0, 3
        observe = env.reset()

        observe, _, _, _ = env.step(1)
        state = preprocess(observe)
        hist = np.stack((state, state, state, state), axis=2)
        hist = np.reshape([hist], (1, 84, 84, 4))
        
        while not done:
            
           
            qvalue = model.predict([hist, np.ones(size).reshape(1, size)])
            action = np.argmax(qvalue[0])         
            observe, reward, done, info = env.step(action)          
            next_state = preprocess(observe)
            next_state = np.reshape([next_state], (1, 84, 84, 1))
            next_hist = np.append(next_state, hist[:, :, :, :3], axis=3)          
            if start_life > info['ale.lives']:
              dead = True
              start_life = info['ale.lives']
            score += reward
            if dead:
              dead = False
              print ("agent is dead")
              history = next_hist
            else:
              history = next_hist 
            global_step += 1
            if done:
                episode_number += 1
                print('episode: {}, score: {}'.format(episode_number, score))
                
    env.close()
    show_video()
test()

agent is dead
agent is dead
agent is dead
episode: 1, score: 285.0
